In [ ]:
import os
import sys
import math
import argparse
import numpy as np
from collections import Counter
import xml.etree.ElementTree as et 

# Keras imports
from keras.models import Model
from keras.optimizers import Adam
from keras.utils.vis_utils import model_to_dot
from keras.layers import Input, Dense, Dropout, AlphaDropout, BatchNormalization, Activation

# ML4CVD Imports
sys.path.append("../ml4cvd")
from plots import plot_ecg
from arguments import parse_args
from recipes import train_multimodal_multitask

%matplotlib inline
import matplotlib.pyplot as plt


In [ ]:
strip_ekgs = {}
ffts = {}
xml_file = '/mnt/disks/data/raw/ecgs/5223300_20205_2_0.xml'

root = et.parse(xml_file).getroot()
for c in root.findall("./StripData/WaveformData"):
    strip_ekgs[c.attrib['lead']] = np.array(list(map(float, c.text.strip().split(','))))
    ffts[c.attrib['lead']] = np.fft.fft(strip_ekgs[c.attrib['lead']])

plot_ecg(strip_ekgs, 'strip')

In [ ]:
for c in root.findall("./RestingECGMeasurements"):
    for child in c:
        print(child.tag, child.text)
        
for d in root.findall("./Interpretation/Diagnosis/DiagnosisText"):
	diagnosis = ''.join(e for e in d.text if e.isalnum() or e == ' ')
	print(diagnosis)

    
rows = int(math.ceil(math.sqrt(len(ffts))))
cols = math.ceil(len(ffts)/rows)
fig, axes = plt.subplots(rows, cols, figsize=(28,24))

for i,k in enumerate(ffts):
    ax = plt.subplot(rows, cols, i+1)
    ax.set_title('Frequency Magnitude '+k)
    freq = np.fft.fftfreq(ffts[k].shape[-1])
    ax.plot(freq, np.abs(ffts[k]))
    ax.set_ylim([-20, 60000])
    ax.set_xlim([-0.06, 0.06])
plt.tight_layout()
plt.show()

In [ ]:
xml_file = '/mnt/disks/data/raw/ecgs/4856206_20205_2_0.xml'

root = et.parse(xml_file).getroot()
for c in root.findall("./StripData/WaveformData"):
    strip_ekgs[c.attrib['lead']] = np.array(list(map(float, c.text.strip().split(','))))
    ffts[c.attrib['lead']] = np.fft.fft(strip_ekgs[c.attrib['lead']])

plot_ecg(strip_ekgs, 'strip_bradycardia')

In [ ]:
root = et.parse(xml_file).getroot()
for d in root.findall("./Interpretation/Diagnosis/DiagnosisText"):
    diagnosis = ''.join(e for e in d.text if e.isalnum() or e == ' ')
    print(diagnosis)

rows = int(math.ceil(math.sqrt(len(ffts))))
cols = math.ceil(len(ffts)/rows)
fig, axes = plt.subplots(rows, cols, figsize=(28,24))

for i,k in enumerate(ffts):
    ax = plt.subplot(rows, cols, i+1)
    ax.set_title('FFT Magnitude '+k)
    freq = np.fft.fftfreq(ffts[k].shape[-1])
    ax.plot(freq, np.abs(ffts[k]))
    ax.set_ylim([-20, 80000])
    ax.set_xlim([-0.08, 0.08])
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(rows, cols, figsize=(28,24))
for i,k in enumerate(ffts):
    ax = plt.subplot(rows, cols, i+1)
    ax.set_title('FFT Magnitude '+k)
    freq = np.fft.fftfreq(ffts[k].shape[-1])
    ax.plot(freq, ffts[k].real, freq, ffts[k].imag)
    ax.set_ylim([-80000, 80000])
    ax.set_xlim([-0.04, 0.04])
plt.tight_layout()
plt.show()